#### Documentation
Please click ***↳ cells hidden*** below to show the documentation for this notebook.
##### License
> This notebook as a work of software is licensed under the terms of the [AGPL-3.0](https://opensource.org/licenses/AGPL-3.0) or later.
##### About this software
> This notebook allows a number of **analysis tools** to be run on a simulated trajectory. A description of each tool is provided in its respective code cell.
>
> The specified **project folder** must contain the final output of a production simulation (e.g. `sim_reference.xtc`), on which the analyses will be performed.
##### Installation
> The installation notebook, [`Build_to_Google_Drive.ipynb`](https://colab.research.google.com/github/bioinfkaustin/gromacs-on-colab/blob/main/Build_to_Google_Drive.ipynb), must be run before using this notebook.

In [ ]:
%%capture
import os
import re
import shutil

#@markdown Specify the location of the **GROMACS project folder** to analyse. It should contain the outputs from a production run, e.g. `sim.gro`, `sim.trr` and/or `sim.xtc`, and so on.
project_folder = "{GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid" #@param {type: "string"}
# default: {GoogleDrive}/GROMACS/7FBF_FABPH_vs_octanoic_acid

#@markdown Provide the unique filename prefix used for the simulation.
output_prefix = "sim" #@param {type: "string"}
# default: sim

#@markdown **After filling in this form, run the notebook by clicking *Runtime -> Run all* in the toolbar.**

# Google Drive
if not os.path.isdir("/content/drive/MyDrive") and project_folder.startswith("{GoogleDrive}"):
  from google.colab import drive
  drive.mount("/content/drive")
if not os.path.isdir("/content/drive/MyDrive"):
  raise RuntimeError("Error: could not connect to Google Drive")

# Methods for parsing and validation
def _path(s, exists=False):
  if "{GoogleDrive}" in s and not s.startswith("{GoogleDrive}"): raise ValueError(f"Error: {{GoogleDrive}} is a path prefix, but appears later: {s}")
  s = s.format(GoogleDrive="/content/drive/MyDrive")
  if exists and not os.path.isdir(s): raise FileNotFoundError(f"Error: folder not found: {s}")
  return os.path.abspath(s)

def parse(s, mandatory=False, path=False, exists=False):
  s = s.strip()
  if mandatory and not s:
    raise ValueError("Error: mandatory setting without value")
  if path:
    s = _path(s, exists=exists)
  return s

# Validate the input values
project_folder = parse(project_folder, mandatory=True, path=True, exists=True)
if not output_prefix:
  raise RuntimeError("Error: an output prefix must be provided")
if not re.match(r"^[0-9a-zA-Z]+$", output_prefix):
  raise RuntimeError(f"Error: special characters are not allowed in output prefix, but got: {output_prefix}")

# Make sure that the notebook is in the start folder
if "START" not in os.environ or not os.environ["START"]:
  %env START={os.getcwd()}
else:
  %cd {os.environ["START"]}

# Use a clean scratch directory for the rest of the run
try:
  shutil.rmtree("scratch")
except FileNotFoundError:
  pass
os.makedirs("scratch")
%cd "scratch"

# Verbosity
#@markdown By default, only relevant output is displayed. To see the full output, including any error messages, enable the `verbose` option below.
verbose=False #@param {type: "boolean"}
if verbose:
  %env VERBOSE=true
else:
  %env VERBOSE=false

In [ ]:
#@markdown In the following cells, applications are downloaded from a **persistent cache** in your Google Drive.
#@markdown This cell sets up the cache folder.
storage = "/content/drive/MyDrive/gromacs-on-colab"
%env STORAGE={storage}

In [ ]:
%%bash
#@markdown **GROMACS** is installed from cache.
if [[ -d "/usr/local/gromacs" ]]; then
  exit 0 # already installed
fi
gromacs_vers="2023.2" #@param {type: "string"}
cache_gromacs="${STORAGE}/gromacs-${gromacs_vers}.tar.gz"
if [[ -s "${cache_gromacs}" ]]; then
  tar -xzf "${cache_gromacs}" -C "/usr/local"
else
  echo "Error: GROMACS installation not found" >&2
  echo "(Have you installed GROMACS to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
%%bash
#@markdown The CHARMM36 forcefield is downloaded from cache.
if [[ -d "${START}/charmm36.ff" ]]; then
  exit 0 # already installed
fi
charmm36_vers="jul2022" #@param {type: "string"}
cache_charmm36="${STORAGE}/charmm36-${charmm36_vers}.tar.gz"
if [[ -s "${cache_charmm36}" ]]; then
  tar -xzf "${cache_charmm36}" -C "${START}"
else
  echo "Error: CHARMM36 forcefield installation not found" >&2
  echo "(Have you installed the CHARMM36 forcefield to your Google Drive?)" >&2
  exit 1
fi

In [ ]:
%%bash -s "$project_folder" "$output_prefix"
project_folder="$1"
output_prefix="$2"

#@markdown Extract the trajectory from the project folder.
if [[ ! -z "$(ls -A)" ]]; then
  exit 0 # already extracted
fi
if [[ ! -d "${project_folder}" ]]; then
  echo "Error: folder not found: ${project_folder}" >&2
  exit 1
fi
pushd "${project_folder}"
cp "grompp.mdp" "conf.gro" "restraint.gro" "index.ndx" "topol.top" "${START}/scratch/" 2> /dev/null

top_level_dir="$(realpath .)"
function get_includes_recursively {
  f="$1"
  sed -E "/^#include/!d; s/^#include +['\"]//; s/['\"]$//" "${f}" | while read -r g; do
    d="$(dirname "${g}")"
    b="$(basename "${g}")"
    pushd "${d}" > /dev/null
    get_includes_recursively "${b}"
    echo "$(realpath --relative-to="${top_level_dir}" "${b}")"
    popd > /dev/null
  done
}
get_includes_recursively "topol.top" | while read -r f; do
  cp --parents "${f}" "${START}/scratch/"
done

if [[ -s "${output_prefix}.part0001.tar.gz" ]]; then
  latest_part="$(ls | egrep '^'"${output_prefix}"'\.part[0-9]+\.tar.gz$' | sort -V | tail -n1 | egrep -o 'part[0-9]+')"
  tar -xzf "${output_prefix}.${latest_part}.tar.gz" -C "${START}/scratch" 2> /dev/null
else
  cp "${output_prefix}".* "${START}/scratch/" 2> /dev/null
fi
cp "${output_prefix}_reference.xtc" "${START}/scratch/" 2> /dev/null
popd

if [[ ! -s "grompp.mdp" || ! -s "conf.gro" || ! -s "topol.top" ]]; then
  echo "Error: essential files not found: grompp.mdp, conf.gro, topol.top" >&2
  exit 1
fi
if [[ ! -s "${output_prefix}.tpr" ]]; then
  echo "Error: simulation parameters not found: ${output_prefix}.tpr" >&2
  exit 1
fi

if [[ ! -s "${output_prefix}_reference.xtc" ]]; then
  source "/usr/local/gromacs/bin/GMXRC.bash"
  [[ -s "${output_prefix}.trr" ]] && trr_frames="$(gmx check -f "${output_prefix}.trr" 2>&1 | awk '$1 ~ /^Coords/ { print $2 }')" || trr_frames=0
  [[ -s "${output_prefix}.xtc" ]] && xtc_frames="$(gmx check -f "${output_prefix}.xtc" 2>&1 | awk '$1 ~ /^Coords/ { print $2 }')" || xtc_frames=0
  if (( $trr_frames == 0 && $xtc_frames == 0 )); then
    echo "Error: output trajectory not found: ${output_prefix}_reference.xtc" >&2
    exit 1
  elif (( $trr_frames >= $xtc_frames )); then
    xtcext="trr"
    ndecarg="-ndec 4"
  else
    xtcext="xtc"
    ndecarg=""
  fi
  gmx trjconv -f "${output_prefix}.${xtcext}" -s "${output_prefix}.tpr" -pbc nojump $ndecarg -o "nojump.xtc" <<< "0"
  gmx trjconv -f "nojump.xtc" -s "${output_prefix}.tpr" -fit progressive $ndecarg -o "progressive.xtc" < <(echo "C-alpha"; echo "0")
  gmx trjconv -f "progressive.xtc" -s "${output_prefix}.tpr" -pbc mol $ndecarg -o "${output_prefix}_reference.xtc" <<< "0"
fi

In [ ]:
#@markdown This cell deliberately throws an error, to interrupt *Run all*.
#@markdown **In the cells below, press the <u>play button</u> (left) to use the specified tool.**
raise StopIteration("Please continue by running the desired cells manually.")

In [ ]:
%%bash -s "$project_folder"
project_folder="$1"

#@markdown **Save analyses to Google Drive.** After using the tools below, run this cell to save outputs (e.g. graphs) to the project folder.
#@markdown (The upload may take a moment. Please verify the files are in your Google Drive before disconnecting from the session.)
cp "dat."* "${project_folder}/"

In [ ]:
%%bash -s "$output_prefix"
exec 3>&1
if $VERBOSE; then
  exec 2>&1
else
  exec >/dev/null 2>/dev/null
fi
output_prefix="$1"

#@markdown **Compute a centroid,** a representative frame within a time range.
rmsd_group="LIG" #@param {type: "string"}
start_ps=0 #@param {type: "integer"}
end_ps=1000 #@param {type: "integer"}

source "/usr/local/gromacs/bin/GMXRC.bash"
[[ -s "index.ndx" ]] && ndxarg="-n index.ndx" || ndxarg=""

( echo "C-alpha"; echo "${rmsd_group}" ) | gmx rms -nomw -s "${output_prefix}.tpr" -f "${output_prefix}_reference.xtc" -b $start_ps -e $end_ps $ndxarg -m "rmsd.xpm"
( echo "System"; echo "System" ) | gmx cluster -s "${output_prefix}.tpr" -f "${output_prefix}_reference.xtc" -b $start_ps -e $end_ps -dm "rmsd.xpm" -cl "clusters.gro" -clid "clust-id.xvg" -sz "clust-size.xvg"

target_frame=$(paste <(fgrep "frame t=" "clusters.gro") <(sed '/^[@#]/d' "clust-size.xvg") | sort -n -k5 | tail -n1 | awk '{ print $3 }')
name="${output_prefix}_centroid_${rmsd_group}_${start_ps}-${end_ps}"
gmx trjconv -s "${output_prefix}.tpr" -f "${output_prefix}_reference.xtc" -dump $target_frame -o "dat.${name}.gro" <<< "0"
gmx trjconv -s "${output_prefix}.tpr" -f "dat.${name}.gro" -o "dat.${name}.pdb" <<< "0"
rm "rmsd.xpm" "clusters.gro" "clust-id.xvg" "clust-size.xvg"

for i in $(seq 5); do echo ""; done
{
  echo "***"
  echo "Made: dat.${name}.gro"
  echo "Made: dat.${name}.pdb"
  echo "***"
} >&3

In [ ]:
%%bash -s "$output_prefix"
output_prefix="$1"
exec 3>&1
if $VERBOSE; then
  exec 2>&1
else
  exec >/dev/null 2>/dev/null
fi

rmsd_group="LIG" #@param {type: "string"}
reference_ps="0, 0-1000" #@param {type: "string"}

source "/usr/local/gromacs/bin/GMXRC.bash"
[[ -s "index.ndx" ]] && ndxarg="-n index.ndx" || ndxarg=""
log="$(mktemp)"
rmsd_groups="$(tr -d " " <<< "${rmsd_group}" | tr "," " ")"

for rmsd_group in $rmsd_groups; do
  ranges_ps="$(tr -d " " <<< "${reference_ps}" | tr "," " ")"
  for range_ps in $ranges_ps; do
    if [[ -z "${range_ps}" || $range_ps == 0 ]]; then
      if [[ -s "conf.gro" ]]; then
        input="conf.gro"
      else
        range_ps=0
      fi
    fi
    if [[ -z "${range_ps}" ]]; then
      :
    elif [[ "${range_ps}" =~ - ]]; then
      input="dat.${output_prefix}_centroid_${rmsd_group}_${range_ps}.gro"
      if [[ ! -s "${input}" ]]; then
        echo "Error: centroid not found for range ${range_ps}: ${input}" >&2
        exit 1
      fi
    else
      frame_ps=$(($range_ps))
      if [[ -z "${frame_ps}" ]]; then
        echo "Error: invalid frame time: ${frame_ps}" >&2
        exit 1
      fi
      input="dat.${output_prefix}_frame_${frame_ps}.gro"
      if [[ ! -s "${input}" ]]; then
        gmx trjconv -s "${output_prefix}.tpr" -f "${output_prefix}_reference.xtc" -dump $frame_ps -o "${input}" <<< "0"
        echo "Made: ${input}" >> "${log}"
      fi
    fi
    ( echo "C-alpha"; echo "${rmsd_group}"; ) | gmx rms -nomw -s "${input}" $ndxarg -f "${output_prefix}_reference.xtc" -o "rmsd.xvg"
    name="${output_prefix}_rmsd_${rmsd_group}_${range_ps}"
    {
      echo "Time_(ns),RMSD_(A)"
      sed '/^[@#]/d' "rmsd.xvg" | awk '{ print $1/1000","$2*10 }'
    } > "dat.${name}.csv"
    echo "Made: dat.${name}.csv" >> "${log}"
    rm "rmsd.xvg"
  done
done

for i in $(seq 5); do echo ""; done
{
  echo "***"
  cat "${log}"
  echo "***"
} >&3
rm "${log}"

In [ ]:
rmsd_groups = "LIG" #@param {type: "string"}
reference_ps = "0, 0-1000" #@param {type: "string"}
threshold_A = 5.5 #@param {type: "number"}
max_t_ns = 0 #@param {type: "number"}

rmsd_groups = rmsd_groups.replace(" ", "").split(",")
ranges_ps = reference_ps.replace(" ", "").split(",")

import pandas as pd
t_ = "Time_(ns)"
y_ = "RMSD_(A)"
groups, ranges, labels, tables = list(), list(), list(), list()

for g in rmsd_groups:
  for r in ranges_ps:
    try:
      df = pd.read_csv(f"dat.{output_prefix}_rmsd_{g}_{r}.csv")
    except:
      continue
    groups.append(g)
    ranges.append(r)
    if "-" in r:
      start = float(r.split("-")[0]) / 1000.
      labels.append(f"vs. {r} ns centroid: {g}")
    else:
      start = float(r) / 1000.
      labels.append(f"vs. {r} ns frame: {g}")
    df = df[(df[t_] >= start)]
    if max_t_ns > 0.:
      df = df[(df[t_] <= max_t_ns)]
    tables.append(df)

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import AutoLocator

def moving_average(df):
  return df.rolling(21, center=True).mean()

plt.figure()
with sns.color_palette("deep", n_colors=len(tables)):
  for l, df in zip(labels, tables):
    plt.plot(df[t_], df[y_], alpha=0.1)
  for l, df in zip(labels, tables):
    smooth = moving_average(df)
    plt.plot(smooth[t_], smooth[y_], label=l)
  plt.legend()
  if threshold_A != 0.:
    plt.axhline(y=threshold_A, color="black", linestyle=":")

plt.title(os.path.basename(project_folder).replace("_", " ").replace("vs ", "vs. "))
plt.xlabel("Time (ns)")
plt.ylabel(f"RMSD of group (A)")

ticker = AutoLocator()
ticker.set_params(steps=[1, 2, 3, 5, 10] if round(df[t_].values[-1]) % 3 == 0 else [1, 2, 5, 10], min_n_ticks=6)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker)

if any(max(df[y_]) > 16. for df in tables):
  ax.set_ylim([0, 16.])

groups_name = "_".join(rmsd_groups)
ranges_name = "_".join(ranges_ps)
name = f"{output_prefix}_rmsd_{groups_name}_{ranges_name}"
plt.gcf().savefig(f"dat.{name}.png")
plt.show()
print("***")
print(f"Made: dat.{name}.png")
print("***")

In [ ]:
%%bash -s "$output_prefix"
exec 3>&1
if $VERBOSE; then
  exec 2>&1
else
  exec >/dev/null 2>/dev/null
fi
output_prefix="$1"

pair_label="A-B" #@param {type: "string"}
atom_a=1234 #@param {type: "integer"}
atom_b=5678 #@param {type: "integer"}

source "/usr/local/gromacs/bin/GMXRC.bash"
cat > "dist.ndx" <<EOS
[ Distance ]
${atom_a} ${atom_b}
EOS

gmx distance -s "${output_prefix}.tpr" -f "${output_prefix}_reference.xtc" -n "dist.ndx" -oall "dist.xvg" <<< "Distance"
name="${output_prefix}_distance_${pair_label}_0"
{
  echo "Time_(ns),RMSD_(A)"
  sed '/^[@#]/d' "dist.xvg" | awk '{ print $1/1000","$2*10 }'
} > "dat.${name}.csv"

for i in $(seq 5); do echo ""; done
{
  echo "***"
  echo "Information: compared these atoms:"
  awk "NR == ${atom_a} + 2" "conf.gro"
  awk "NR == ${atom_b} + 2" "conf.gro"
  echo "Made: dat.${name}.csv"
  echo "***"
} >&3

In [ ]:
pair_labels = "A-B" #@param {type: "string"}
threshold_A = 0. #@param {type: "number"}
max_t_ns = 0 #@param {type: "number"}

pair_labels = pair_labels.replace(" ", "").split(",")

import pandas as pd
t_ = "Time_(ns)"
y_ = "RMSD_(A)"
labels, tables = list(), list()

for p in pair_labels:
  try:
    df = pd.read_csv(f"dat.{output_prefix}_distance_{p}_0.csv")
  except:
    continue
  labels.append(p)
  if max_t_ns > 0.:
    df = df[(df[t_] <= max_t_ns)]
  tables.append(df)

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import AutoLocator

def moving_average(df):
  return df.rolling(21, center=True).mean()

plt.figure()
with sns.color_palette("deep", n_colors=len(tables)):
  for l, df in zip(labels, tables):
    plt.plot(df[t_], df[y_], alpha=0.1)
  for l, df in zip(labels, tables):
    smooth = moving_average(df)
    plt.plot(smooth[t_], smooth[y_], label=l)
  plt.legend()
  if threshold_A != 0.:
    plt.axhline(y=threshold_A, color="black", linestyle=":")

plt.title(os.path.basename(project_folder).replace("_", " ").replace("vs ", "vs. "))
plt.xlabel("Time (ns)")
plt.ylabel(f"Distance of pair of atoms (A)")

ticker = AutoLocator()
ticker.set_params(steps=[1, 2, 3, 5, 10] if round(df[t_].values[-1]) % 3 == 0 else [1, 2, 5, 10], min_n_ticks=6)
ax = plt.gca()
ax.xaxis.set_major_locator(ticker)

pairs_name = "_".join(pair_labels)
name = f"{output_prefix}_distance_{pairs_name}_0"
plt.gcf().savefig(f"dat.{name}.png")
plt.show()
print("***")
print(f"Made: dat.{name}.png")
print("***")

In [ ]:
%%bash -s "$output_prefix"
output_prefix="$1"
exec 3>&1
if $VERBOSE; then
  exec 2>&1
else
  exec >/dev/null 2>/dev/null
fi
set +H

energy_group="LIG" #@param {type: "string"}
start_ps=0 #@param {type: "integer"}
end_ps=1000 #@param {type: "integer"}

source "/usr/local/gromacs/bin/GMXRC.bash"

if [[ ! -s "index.ndx" ]]; then
  gmx make_ndx -f "conf.gro" -o "index.ndx" <<< "q"
fi
if ! fgrep -q "Surround" "index.ndx"; then
  group_id=$(fgrep '[' "index.ndx" | wc -l)
  { echo "! \"${energy_group}\""; echo "name ${group_id} Surround"; echo "q"; } | gmx make_ndx -f "conf.gro" -n "index.ndx" -o "index.ndx"
fi

( cat "grompp.mdp"; echo "energygrps = Surround ${energy_group}" ) > "lie_protein.mdp"
[[ ! -s "restraint.gro" ]] && cp "conf.gro" "restraint.gro"
gmx grompp -f "lie_protein.mdp" -o "lie_protein.tpr" -c "conf.gro" -p "topol.top" -r "restraint.gro" -n "index.ndx" -maxwarn 999
rerun="${output_prefix}_reference.xtc"
gmx trjconv -s "${output_prefix}.tpr" -f "${rerun}" -e $end_ps -o "rerun.trr" <<< "0"
gmx mdrun -deffnm "lie_protein" -rerun "rerun.trr" || exit $?

read -r protein_qq protein_lj < <({
  echo "Coul-SR:Surround-${energy_group}";
  echo "LJ-SR:Surround-${energy_group}";
  echo;
} | gmx energy -f "lie_protein.edr" -b $start_ps -e $end_ps 2>/dev/null | tail -n2 | awk '{ print $2 }' | tr "\n" " ")
protein_total="$(perl -le "print(${protein_qq} + ${protein_lj})")"

for i in $(seq 5); do echo ""; done
{
  echo "***"
  echo "Total interaction energy (ligand with protein): ${protein_total}"
  echo "Electrostatics term: ${protein_qq}"
  echo "Van der Waals term: ${protein_lj}"
  echo "***"
} >&3

In [ ]:
%%bash -s "$output_prefix"
output_prefix="$1"
exec 3>&1
if $VERBOSE; then
  exec 2>&1
else
  exec >/dev/null 2>/dev/null
fi
set +H

energy_group="LIG" #@param {type: "string"}

source "/usr/local/gromacs/bin/GMXRC.bash"
[[ -s "index.ndx" ]] && ndxarg="-n index.ndx" || ndxarg=""

gmx trjconv -s "${output_prefix}.tpr" -f "${output_prefix}_reference.xtc" $ndxarg -dump 1000 -o "ligand.gro" <<< "${energy_group}"
gmx editconf -f "ligand.gro" -o "out.gro" -c -d 1.0 -bt "dodecahedron"
mv "out.gro" "ligand.gro"
sed -E '/\[\s*molecules\s*\]/q' "topol.top" > "ligand.top"
echo "${energy_group} 1" >> "ligand.top"
awk '! x && $0 ~ /^#include/ { x=1; print "#include \"'"${START}"'/charmm36.ff/forcefield.itp\"" }; $0 !~ /forcefield.itp/ { print }' "ligand.top" > "ligand.top.tmp"
mv "ligand.top.tmp" "ligand.top"
sed 's/SOL /TIP3/g' "/usr/local/gromacs/share/gromacs/top/spc216.gro" > "TIP3.gro"
gmx solvate -cp "ligand.gro" -cs "TIP3.gro" -o "out.gro" -p "ligand.top"
mv "out.gro" "ligand.gro"

cat > "ligand_min.mdp" <<EOS
integrator = steep
emtol = 1000.0
nsteps = 5000
cutoff-scheme = Verlet
nstlist = 10
coulombtype = PME
rcoulomb = 1.2
vdwtype = Cut-off
vdw-modifier = Force-switch
rvdw-switch = 1.0
rvdw = 1.2
constraints = h-bonds
constraint-algorithm = LINCS
EOS
gmx grompp -f "ligand_min.mdp" -o "ligand_min.tpr" -c "ligand.gro" -p "ligand.top" -maxwarn 999
gmx genion -s "ligand_min.tpr" -o "ligand_with_ions.gro" -p "ligand.top" -pname "POT" -nname "CLA" -neutral <<< "TIP3"
mv "ligand_min.gro" "ligand_min.gro.bak"
mv "ligand_with_ions.gro" "ligand_min.gro"
gmx grompp -f "ligand_min.mdp" -o "ligand_min.tpr" -c "ligand_min.gro" -p "ligand.top" -maxwarn 999
gmx mdrun -v -stepout 1000 -deffnm "ligand_min" || exit $?

cat > "ligand_eq.mdp" <<EOS
integrator = md
dt = 0.001
nsteps = 100000
comm-mode = Linear
cutoff-scheme = Verlet
nstlist = 20
coulombtype = PME
rcoulomb = 1.2
vdwtype = Cut-off
vdw-modifier = Force-switch
rvdw-switch = 1.0
rvdw = 1.2
tcoupl = Berendsen
tc-grps = System
tau-t = 1.0
ref-t = 303.15
pcoupl = Berendsen
pcoupltype = isotropic
tau-p = 5.0
compressibility = 4.5e-5
ref-p = 1.0
refcoord-scaling = com
constraints = h-bonds
constraint-algorithm = LINCS
continuation = yes
EOS
gmx grompp -f "ligand_eq.mdp" -o "ligand_eq.tpr" -c "ligand_min.gro" -p "ligand.top" -maxwarn 999
gmx mdrun -v -stepout 1000 -deffnm "ligand_eq" || exit $?

cat > "ligand_prod.mdp" <<EOS
integrator = md
dt = 0.002
nsteps = 50000
comm-mode = Linear
nstxout = 5000
nstvout = 5000
cutoff-scheme = Verlet
nstlist = 20
coulombtype = PME
rcoulomb = 1.2
vdwtype = Cut-off
vdw-modifier = Force-switch
rvdw-switch = 1.0
rvdw = 1.2
tcoupl = V-Rescale
tc-grps = System
tau-t = 1.0
ref-t = 303.15
pcoupl = C-Rescale
pcoupltype = isotropic
tau-p = 5.0
compressibility = 4.5e-5
ref-p = 1.0
constraints = h-bonds
constraint-algorithm = LINCS
continuation = yes
EOS
gmx grompp -f "ligand_prod.mdp" -o "ligand_prod.tpr" -c "ligand_eq.gro" -p "ligand.top" -maxwarn 999
gmx mdrun -v -stepout 1000 -deffnm "ligand_prod" || exit $?

echo "q" | gmx make_ndx -f "ligand_min.gro" -o "ligand.ndx"
group_id=$(fgrep '[' "ligand.ndx" | wc -l)
{ echo "! \"${energy_group}\""; echo "name ${group_id} Surround"; echo "q"; } | gmx make_ndx -f "ligand_min.gro" -n "ligand.ndx" -o "ligand.ndx"
( cat "ligand_prod.mdp"; echo "energygrps = ${energy_group} Surround" ) > "lie_ligand.mdp"
gmx grompp -f "lie_ligand.mdp" -o "lie_ligand.tpr" -c "ligand_min.gro" -p "ligand.top" -n "ligand.ndx" -maxwarn 999
gmx mdrun -deffnm "lie_ligand" -rerun "ligand_prod.trr" || exit $?

read -r ligand_qq ligand_lj < <({
  echo "Coul-SR:${energy_group}-Surround";
  echo "LJ-SR:${energy_group}-Surround";
  echo;
} | gmx energy -f "lie_ligand.edr" 2>/dev/null | tail -n2 | awk '{ print $2 }' | tr "\n" " ")
ligand_total="$(perl -le "print(${ligand_qq} + ${ligand_lj})")"

for i in $(seq 5); do echo ""; done
{
  echo "***"
  echo "Total interaction energy (ligand with solvent): ${ligand_total}"
  echo "Electrostatics term: ${ligand_qq}"
  echo "Van der Waals term: ${ligand_lj}"
  echo "***"
} >&3